In [3]:
import pandas as pd
from datetime import datetime
import dateutil.relativedelta
# import pypyodbc as odbc

# Test database

In [7]:
# drive_name = 'SQL SERVER'
# server_name = 'DESKTOP-29355SM\SQLEXPRESS'
# database_name = 'Bank'

# connection_string = f"""
#     DRIVER={{{drive_name}}};
#     SERVER={server_name};
#     DATABASE={database_name};
#     Trust_Connection=yes;
# """

# conn = odbc.connect(connection_string)

In [9]:
# cursor = conn.cursor()

In [13]:
# cursor.execute("""
# Create table Persons (
#     PersonID int,
#     LastName varchar(255),
#     FirstName varchar(255),
#     Address varchar(255),
#     City varchar(255)
# )
# """)

In [ ]:
# df = pd.read_sql('Select * from account', conn)

In [14]:
# conn.commit()

# Function

In [48]:
def extract_date_review(text_date, crawl_date): # Biến đổi khoảng thời gian (1 tháng trước, 2 ngày trước) thằnh ngày cụ thể và trả về datetime
    sub_text = text_date.split(' ')
    date_cmt = ''

    if 'năm' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(t))
    elif 'tháng' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(months=int(t))
    elif 'tuần' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(weeks==int(t))
    elif 'ngày' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(days=int(t))
    else:
        date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date()

    return date_cmt

# Main

In [1]:
# product = input()
product = 'iphone 15 pro max'

# Tiki

#### Product info

In [8]:
tiki_product_info = pd.read_csv(r'..\Data\Tiki\%s_info.csv'%('_'.join(product.split(' '))))

In [14]:
# Lấy những cột cần thiết
tiki_product_info = tiki_product_info[['Name', 'Brand', 'Quantity', 'Reviews count', 'Star rating', 
                                       'Price', 'Crawl date', 'Shop name', 'Shop rating']]

In [15]:
tiki_product_info.dtypes

Name              object
Brand             object
Quantity           int64
Reviews count      int64
Star rating      float64
Price             object
Crawl date        object
Shop name         object
Shop rating      float64
dtype: object

In [80]:
# Thêm cột "Platform ID"
tiki_product_info['Platform ID'] = 'PL01'

In [81]:
# Loại bỏ dấu "." trong cột "Price" và đổi định dạng thành int
tiki_product_info['Price'] = tiki_product_info['Price'].str.replace('.', '', regex=True).astype(int)

In [82]:
tiki_product_info

,Name,Brand,Quantity,Reviews count,Star rating,Price,Crawl date,Shop name,Shop rating,Platform ID
0,Apple iPhone 15 Pro Max,Apple,691,96,5.0,31990000,2024-01-11,Tiki Trading,4.7,PL01
1,Apple iPhone 15 Pro Max,Apple,726,100,5.0,31990000,2024-01-18,Tiki Trading,4.7,PL01
2,Apple iPhone 15 Pro Max,Apple,740,103,5.0,31990000,2024-01-27,Tiki Trading,4.7,PL01


In [83]:
tiki_product_info.dtypes

Name              object
Brand             object
Quantity           int64
Reviews count      int64
Star rating      float64
Price              int32
Crawl date        object
Shop name         object
Shop rating      float64
Platform ID       object
dtype: object

#### Reviews

In [50]:
tiki_reviews = pd.read_csv(r'..\Data\Tiki\%s_reviews.csv'%('_'.join(product.split(' '))))

In [51]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Crawl date,Shop name
0,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
1,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
2,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
3,NGÔ QUỐC KHÁNH,Tiki giao lúc 1h sáng. Áp mã 500K và giảm 2tr ...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
4,Hung Trieu,Rất tin tưởng Tiki khi đặt mua điện thoại tại ...,Cực kì hài lòng,Đánh giá vào 2 tháng trước,2024-01-11,Tiki Trading
...,...,...,...,...,...,...
294,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,Rất không hài lòng,Đánh giá vào 2 tháng trước,2024-01-27,Tiki Trading
295,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,Rất không hài lòng,Đánh giá vào 3 tháng trước,2024-01-27,Tiki Trading
296,Thu Nga,Chưa thấy hoàn astra cho mình,Không hài lòng,Đánh giá vào 3 tháng trước,2024-01-27,Tiki Trading
297,Nguyễn Quang,NaN,Hài lòng,Đánh giá vào 2 tháng trước,2024-01-27,Tiki Trading


In [52]:
# Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
actual_date_list = []

for num in range(len(tiki_reviews)):
    text_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][0]
    crawl_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][1]

    actual_date = extract_date_review(text_date, crawl_date)
    actual_date_list.append(actual_date)

tiki_reviews['Rating date'] = actual_date_list
tiki_reviews.drop(columns=['Crawl date'], inplace=True)

In [64]:
# Biến đổi cột "Rating" thành thang đo likert
rating_likert = {'Rất không hài lòng': 1, 'Không hài lòng': 2, 'Bình thường': 3, 'Hài lòng': 4, 'Cực kì hài lòng': 5}

for num in range(len(tiki_reviews)):
    rating = tiki_reviews.loc[num, 'Rating']
    for k, v in rating_likert.items():
        if rating == k:
            tiki_reviews.loc[num, 'Rating'] = v

In [73]:
# Loại bỏ những dòng trùng "Reviewer name" và "Content"
tiki_reviews.drop_duplicates(subset=['Reviewer name', 'Content'], keep='last', inplace=True, ignore_index=True)

In [76]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Shop name
0,Lý Nguyen,😂. Haha,5,2023-12-23,Tiki Trading
1,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,1,2023-12-11,Tiki Trading
2,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",5,2023-09-27,Tiki Trading
3,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,5,2023-09-27,Tiki Trading
4,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,5,2023-09-27,Tiki Trading
...,...,...,...,...,...
96,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,1,2023-11-27,Tiki Trading
97,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,1,2023-10-27,Tiki Trading
98,Thu Nga,Chưa thấy hoàn astra cho mình,2,2023-10-27,Tiki Trading
99,Nguyễn Quang,NaN,4,2023-11-27,Tiki Trading


# Lazada

# Websites

#### Tinhte _Uncompleted

In [78]:
tt_keyword = [['Iphone15pm','P001'],['SamsungS23u','P002']]

In [81]:
df = pd.DataFrame()

for i in range(len(tt_keyword)):
    
    name_info= pd.read_csv(f'..\Data\TinhTe\TinhTe_{tt_keyword[i][0]}_InfoPosts.csv')
    name_cmt = pd.read_csv(f'..\Data\TinhTe\TinhTe_{tt_keyword[i][0]}_Comments.csv')

    #Rename columns following the required format in the Database
    name_info.rename(columns={'Link':'SMReviewID','Author':'ReviewerName','Post Date':'DateReview','Comments Count':'CommentCount','Content':'Review'},inplace=True)
        # covert date column in name_cmt into right format
        # use a function
        
    name_cmt.rename(columns={'Link':'SMReviewID','Date':'DateReview','Comment':'Review'},inplace=True)
    
    # Add PlatformID column
    name_info['PlatformID'] = 'PL05'
    name_cmt['PlatformID'] = 'PL05'

    # Add ProductID column
    name_info['ProductID'] = tt_keyword[i][1]
    name_cmt['ProductID'] = tt_keyword[i][1]

    # Add TypeReview column
    name_info['TypeReview'] = 'Post'
    name_cmt['TypeReview'] = 'Comment'
    df = pd.concat([df,name_info,name_cmt],ignore_index=True)
    


4626
10724


In [85]:
cmt = pd.read_csv('../Data/TinhTe/TinhTe_Iphone15pm_Comments.csv')

In [86]:
cmt

,Link,Date,Comment
0,https://tinhte.vn/thread/nang-cap-len-iphone-1...,3 tháng,Bài viết hay. \r\nThật sự cầm cái 14prm trên t...
1,https://tinhte.vn/thread/nang-cap-len-iphone-1...,3 tháng,"cầm cả năm thì cũng quen, nhưng nặng thì vẫn l..."
2,https://tinhte.vn/thread/nang-cap-len-iphone-1...,3 tháng,Cầm 14pm rồi cầm 15pm thấy khác ngay và luôn đ...
3,https://tinhte.vn/thread/nang-cap-len-iphone-1...,3 tháng,Mình cầm cả hai rồi thấy ko vấn đề về độ nặng....
4,https://tinhte.vn/thread/nang-cap-len-iphone-1...,3 tháng,"Mình ms cầm là thấy ngay, đưa cho tất cả đồng ..."
...,...,...,...
4526,https://tinhte.vn/thread/iphone-15-co-the-duoc...,6 tháng,Kinh tế năm nay khó khăn mà Apple tăng sản lượ...
4527,https://tinhte.vn/thread/iphone-15-co-the-duoc...,6 tháng,Ip 13 của mình bị bể mặt lưng biết đến bao h b...
4528,https://tinhte.vn/thread/iphone-15-co-the-duoc...,6 tháng,"Nghe có vẻ sang xịn đấy,nhưng để bỏ ra 50 củ k..."
4529,https://tinhte.vn/thread/iphone-15-co-the-duoc...,6 tháng,Rồi có thêm đặc sản phải câu dây đồng khi hết ...


# Youtube